In [160]:
from typing import Tuple, Optional
from datetime import datetime, timedelta
import math

In [166]:
def get_quadrant(course):
    return course // 90 + 1

def degree_2_radian(angle):
    return (angle * math.pi) / 180

class Vessel:
    """Een klasse die een vaartuig beschrijft."""

    def __init__(self, name: str, position: Tuple[float, float], speed: float, true_course: float):
        self.name = name
        self.position = position #(lat, long)
        self.speed = speed
        self.true_course = true_course

    def __str__(self):
        return (f"Name: {self.name}, position: {self.position}, "
                f"Speed: {self.speed} knots, Course: {self.true_course}°")
    
    def calculate_next_position(self, t1: datetime, t2: datetime):
        """Plane sailing methode"""

        time_difference = t2 - t1
        distance = (time_difference.total_seconds() / 3600) * self.speed

        departure = distance * math.sin(degree_2_radian(self.true_course)) #NM
        delta_latitude = distance * math.cos(degree_2_radian(self.true_course)) #NM

        delta_latitude = delta_latitude / 60 #degrees

        mid_latitude = self.position[0] + delta_latitude/2

        delta_longitude = departure / math.cos(degree_2_radian(mid_latitude)) #NM
        delta_longitude = delta_longitude / 60

        latitude = self.position[0] + delta_latitude
        longitude = self.position[1] + delta_longitude

        if longitude > 180.0:
            longitude = longitude - 360.0
        elif longitude < -180.0:
            longitude = longitude + 360

        if latitude > 90.0: 
            latitude = latitude - 180.0
        elif latitude < -90.0:
            latitude = latitude + 180.0

        return (latitude, longitude)
    
    def change_position(self, latitude: Optional[float] = None, longitude: Optional[float] = None, position: Optional[Tuple[float, float]] = None):
        if position != None:
            self.position = position
        else:
            if latitude is not None and longitude is not None:
                self.position = (latitude, longitude)
            else:
                raise ValueError('Either lat and long or position!')
            
            

In [167]:
MSC_Maya = Vessel(name='MSC Maya', position=(-40.41666667, 175.8333333333), speed=50, true_course=50)

In [168]:
new_position = MSC_Maya.calculate_next_position(datetime.now() - timedelta(hours=10), datetime.now())
print(new_position)

MSC_Maya.change_position(position=new_position)
print(MSC_Maya)


(-35.06010325490395, -176.09434323113155)
Name: MSC Maya, position: (-35.06010325490395, -176.09434323113155), Speed: 50 knots, Course: 50°
